#Destinos: París y Berlín

In [1]:
import asyncio
import json
import random
import string
from typing import List, TypedDict

import requests
!pip install loguru
from loguru import logger as log

# Importamos las librerías que necesitamos
from typing import List, TypedDict

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
!pip install httpx
import httpx
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
!pip install httpx[http2]

In [6]:
class DatosLocalizacion (TypedDict):
    nombrelocaliz: str
    url: str
    url_hoteles: str 
    url_atracciones: str
    url_restaurantes: str
    tipodesitio: str
    latitud: float
    longitud: float 


In [14]:
async def scrape_location_data (query:str, client:httpx.AsyncClient) -> List[DatosLocalizacion]:
    log.info(f"scraping data {query}")
    payload = [
             {
         "variables" : {
         "request": {
             "query": query,
             "limit": 10,
             "scope": "WORLDWIDE",
             "locale": "en-US",
             "scopeGeoId": 1,
             "searchCenter": "None",
             "locationtypes": [
                 "GEO",
                 "AIRPORT",
                 "ACCOMODATION",
                 "ATTRACTION",
                 "ATTRACTION_PRODUCT",
                 "EATERY",
                 "NEIGHBORHOOD",
                 "AIRLINE",
                 "SHOPPING",
                 "UNIVERSITY",
                 "GENERAL_HOSPITAL"
                 "PORT",
                 "FERRY",
                 "CORPORATION",
                 "VACATION_RENTAL",
                 "SHIP",
                 "CRUISE_LINE",
                 "CAR_RENTAL_OFFICE",
                    ],
             "userId":None,
             "context": {},
             "enabledFeatures": ["articles"],
             "includeRecent": True,
               }
         },
         "query" : "84b17ed122fbdbd4",
         "extensions" : {"preRegisteredQueryId": "84b17ed122fbdbd4"},
     }     
 ]
 # we need to generate a random request ID for this request to succeed
    random_request_id = "".join(
        random.choice(string.ascii_lowercase + string.digits) for i in range(180)
    )
    headers = {
        "X-Requested-By": random_request_id,
        "Referer": "https://www.tripadvisor.com/Hotels",
        "Origin": "https://www.tripadvisor.com",
    }
    result = await client.post(
        url="https://www.tripadvisor.com/data/graphql/ids",
        json=payload,
        headers=headers,
    )
    data = json.loads(result.content)
    results = data[0]["data"]["Typeahead_autocomplete"]["results"]
    results = [r["details"] for r in results]  # strip metadata
    log.info(f"found {len(results)} results")
    return results

# To avoid being instantly blocked we'll be using request headers that
# mimic Chrome browser on Windows
BASE_HEADERS = {
    "authority": "www.tripadvisor.com",
    "accept-language": "en-US,en;q=0.9",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}
# start HTTP session client with our headers and HTTP2
client = httpx.AsyncClient(
    http2=True,  # http2 connections are significantly less likely to get blocked
    headers=BASE_HEADERS,
    timeout=httpx.Timeout(150.0),
    limits=httpx.Limits(max_connections=5),
)


async def run():
    result = await scrape_location_data("Malta", client)
    print(json.dumps(result, indent=2))


if __name__ == "__main__":
    asyncio.run(run())

RuntimeError: asyncio.run() cannot be called from a running event loop